# DistilBERT Testing Notebook

In [1]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [6]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from constants import (
    TARGET_SPARSITY_LOW, TARGET_SPARSITY_MID, TARGET_SPARSITY_HIGH,
    BATCH_SIZE_CNN, BATCH_SIZE_VIT, BATCH_SIZE_LLM,
    EPOCHS_SMALL_MODEL, EPOCHS_LARGE_MODEL, EPOCHS_VIT
)
from utils import get_device, get_num_workers, load_weights, print_statistics
from unstructured_pruning import check_model_sparsity, check_sparsity_distribution
from trainer import TrainingArguments, Trainer
from bacp import BaCPTrainingArguments, BaCPTrainer

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 

In [7]:
DEVICE = get_device()
NUM_WORKERS = get_num_workers()
print("Using device:", DEVICE)
print("Using", NUM_WORKERS, "workers")

Using device: cuda
Using 288 workers


# DistilBERT

In [8]:
MODEL_NAME = "distilbert-base-uncased"
MODEL_TASK = "wikitext2"
TRAIN = True

## Baseline Accuracies

In [9]:
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-5),
    scheduler_type='linear_with_warmup',
    epochs=50,
    learning_type="baseline",
    db=False,
)
trainer = Trainer(training_args=training_args)
if False:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Optimizer type w/ learning rate: (adamw, 5e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] Linear scheduler initialized with warmup steps: 355 and total steps: 3550
[TRAINER] Pruning not initialized
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.0



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.37%
  Perplexity:   5.601

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.0000 (0.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        baseline
  Batch Size:           64
  Learning Rate:        5e-05
  Optimizer:            adamw
  Epochs:               50

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## Pruning Accuracies

### Magnitude Prune

In [29]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if False:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.95_pruning.pt
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     10.14%
  Perplexity:   5466.271

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        2e-05
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



{'input_ids': tensor([[  101,   102,   101,  ...,   103, 15046,  1999],
        [ 2526,  2004,   103,  ...,  1999,   103,  8595],
        [ 1010,  2040, 22073,  ...,  1996,  3166,  2043],
        ...,
        [ 2244,   103,  1012,  ...,  1011,  1030,  9152],
        [ 1030,  1011,  1030,  ...,  2743, 17318,  2247],
        [ 2049,  2670,  2918,  ...,   103,  6445,  2094]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  2198,  -100,  -100],
        [ -100,  -100,  1000,  ...,  -100, 20325,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        ...,
        [ -100,  1019,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  1996,  -100,  -100]])}

In [31]:
for data in trainer.testloader:
    print(data)
    break

{'input_ids': tensor([[  101,   102,   101,  ...,   103, 15046,  1999],
        [ 2526,  2004,   103,  ...,  1999,   103,  8595],
        [ 1010,  2040, 22073,  ...,  1996,  3166,  2043],
        ...,
        [ 2244,   103,  1012,  ...,  1011,  1030,  9152],
        [ 1030,  1011,  1030,  ...,  2743, 17318,  2247],
        [ 2049,  2670,  2918,  ...,   103,  6445,  2094]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  2198,  -100,  -100],
        [ -100,  -100,  1000,  ...,  -100, 20325,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        ...,
        [ -100,  1019,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  1996,  -100,  -100]])}


In [32]:
for data in trainer.valloader:
    print(data)
    break

{'input_ids': tensor([[  101,   102,   101,  ...,  2137,   103,  1012],
        [ 1996,   103,  2427,  ...,  5046, 26158,  2015],
        [  103,   103,   103,  ...,  1010,  1998,  2087],
        ...,
        [ 2863,   103, 23593,  ...,   103,  1005,  1055],
        [ 2190,  2434,   103,  ...,  2806,  6186,  2299],
        [  103,  6232, 10057,  ...,  1010,   103,   103]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  -100,  2271,  -100],
        [ -100,  2048,  -100,  ...,  -100,  -100,  -100],
        [ 5939, 22747, 24876,  ...,  -100,  -100,  2087],
        ...,
        [ -100,  1010,  -100,  ...,  7579,  -100,  -100],
        [ -100,  -100,  2573,  ...,  -100,  -100,  -100],
        [ 2008,  -100,  -100,  ...,  -100,  1996,  2299]])}


In [34]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if False:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.97_pruning.pt
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     8.65%
  Perplexity:   7899.856

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        2e-05
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [24]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/magnitude_pruning/0.99/run_3.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Recovery epoch [1/10]: Avg Loss: 1.5665 | Avg Accuracy: 64.28 | Model Sparsity: 0.4831
Avg Perplexity: 5.570

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 1.5645 | Avg Accuracy: 64.02 | Model Sparsity: 0.4831
Avg Perplexity: 5.635



...

Recovery epoch [7/10]: Avg Loss: 1.5284 | Avg Accuracy: 64.37 | Model Sparsity: 0.99
Avg Perplexity: 5.589



Recovery epoch [8/10]: Avg Loss: 1.5233 | Avg Accuracy: 64.11 | Model Sparsity: 0.99
Avg Perplexity: 5.573



Recovery epoch [9/10]: Avg Loss: 1.5195 | Avg Accuracy: 63.85 | Model Sparsity: 0.99
Avg Perplexity: 5.724



Recovery epoch [10/10]: Avg Loss: 1.5156 | Avg Accuracy: 64.24 | Model Sparsity: 0.99
Avg Perplexity: 5.508

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.68%
  Perplexity:   5.528

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        2e-05
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### SNIP-it Prune

In [25]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/snip_pruning/0.95/run_2.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Recovery epoch [1/10]: Avg Loss: 1.5661 | Avg Accuracy: 64.13 | Model Sparsity: 0.4636
Avg Perplexity: 5.748

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 1.5696 | Avg Accuracy: 63.57 | Model Sparsity: 0.4636
Avg Perplexity: 5.869



...

Recovery epoch [7/10]: Avg Loss: 1.5204 | Avg Accuracy: 64.09 | Model Sparsity: 0.95
Avg Perplexity: 5.650



Recovery epoch [8/10]: Avg Loss: 1.5291 | Avg Accuracy: 64.12 | Model Sparsity: 0.95
Avg Perplexity: 5.668



Recovery epoch [9/10]: Avg Loss: 1.5275 | Avg Accuracy: 63.55 | Model Sparsity: 0.95
Avg Perplexity: 5.806



Recovery epoch [10/10]: Avg Loss: 1.5330 | Avg Accuracy: 64.33 | Model Sparsity: 0.95
Avg Perplexity: 5.674

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.52%
  Perplexity:   5.533

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        2e-05
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [26]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/snip_pruning/0.97/run_2.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Recovery epoch [1/10]: Avg Loss: 1.5746 | Avg Accuracy: 63.63 | Model Sparsity: 0.4734
Avg Perplexity: 5.860

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 1.5677 | Avg Accuracy: 64.15 | Model Sparsity: 0.4734
Avg Perplexity: 5.687

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 1.5226 | Avg Accuracy: 64.46 | Model Sparsity: 0.97
Avg Perplexity: 5.542



Recovery epoch [8/10]: Avg Loss: 1.5289 | Avg Accuracy: 64.27 | Model Sparsity: 0.97
Avg Perplexity: 5.612



Recovery epoch [9/10]: Avg Loss: 1.5189 | Avg Accuracy: 64.33 | Model Sparsity: 0.97
Avg Perplexity: 5.601



Recovery epoch [10/10]: Avg Loss: 1.5367 | Avg Accuracy: 63.69 | Model Sparsity: 0.97
Avg Perplexity: 5.740

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.76%
  Perplexity:   5.579

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        2e-05
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [27]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/snip_pruning/0.99/run_2.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Recovery epoch [1/10]: Avg Loss: 1.5648 | Avg Accuracy: 63.60 | Model Sparsity: 0.4831
Avg Perplexity: 5.803

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 1.5668 | Avg Accuracy: 64.59 | Model Sparsity: 0.4831
Avg Perplexity: 5.585

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 1.5282 | Avg Accuracy: 64.48 | Model Sparsity: 0.99
Avg Perplexity: 5.567

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 1.5332 | Avg Accuracy: 63.99 | Model Sparsity: 0.99
Avg Perplexity: 5.799



Recovery epoch [9/10]: Avg Loss: 1.5320 | Avg Accuracy: 64.21 | Model Sparsity: 0.99
Avg Perplexity: 5.556



Recovery epoch [10/10]: Avg Loss: 1.5250 | Avg Accuracy: 64.21 | Model Sparsity: 0.99
Avg Perplexity: 5.706

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.78%
  Perplexity:   5.558

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        2e-05
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### WandA Prune

In [28]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 2e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized


AttributeError: 'NoneType' object has no attribute 'dim'

In [0]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 2e-5),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

## BaCP Accuracies

### Magnitude Pruning

In [0]:
trained_model_path = f"/dbfs/research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [0]:
trained_model_path = f"/dbfs/research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [0]:
trained_model_path = f"/dbfs/research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

### Movement Prune

In [0]:
model_name = "distilbert-base-uncased"
model_task = "wikitext2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_{model_task}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="movement_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [0]:
model_name = "distilbert-base-uncased"
model_task = "wikitext2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_{model_task}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="movement_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [0]:
model_name = "distilbert-base-uncased"
model_task = "wikitext2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_{model_task}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="movement_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if False:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

### WandA Prune

In [0]:
trained_model_path = f"/dbfs/research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [0]:
trained_model_path = f"/dbfs/research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [0]:
trained_model_path = f"/dbfs/research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type='adamw',
    learning_rate=1e-3,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print(metrics)